# 1NN Using light sensor

### Setup

In [8]:
import rpyc
import numpy as np
from IPython import display

conn = rpyc.classic.connect("ev3dev.local") # host name or IP address of the EV3
ev3 = conn.modules['ev3dev2']      # import ev3dev2.ev3 remotely

legosensors=conn.modules['ev3dev2.sensor.lego']
sensor_controls=conn.modules['ev3dev2.sensor']
sound=conn.modules['ev3dev2.sound']
led_controls=conn.modules['ev3dev2.led']

led=led_controls.Leds()
sound=sound.Sound()
        

In [10]:
out1=sensor_controls.INPUT_1
out2=sensor_controls.INPUT_2
out3=sensor_controls.INPUT_3

touch = legosensors.TouchSensor(out3)
touchstop = legosensors.TouchSensor(out2)
color =legosensors.ColorSensor(out1)


In [3]:
def dist(color1,color2):
    col1=list(color1)
    col2=list(color2)
    return(np.sqrt((col1[0]-col2[0])**2+(col1[1]-col2[1])**2+(col1[2]-col2[2])**2))


### Data

In [ ]:
while True:
    if touch.is_pressed:
        colordict[color.color_name]=color.raw
    if touchstop.is_pressed:
        break

In [4]:
colordict={
    "Black":(0,0,0),
    "Blue":(0,0,255),
    "Red":(255,0,0),
    "Green":(0,255,0),
    "Yellow":(255,255,0),
    "White":(255,255,255)
}

### Execution

In [11]:
while True:
    if touch.is_pressed:
        color_raw=color.raw
        bestdist=500
        bestcolor="BLACK"
        distdict={}
        for dictcolor,raw in colordict.items():
            distdict[dictcolor]=dist(raw,color_raw)
            if dist(raw,color_raw)<bestdist:
                bestdist=dist(raw,color_raw)
                bestcolor=dictcolor
                
        display.clear_output(wait=True)
        for color_name,distance in distdict.items():
            display.display(color_name,distance)
        sound.speak(bestcolor)
    

'Black'

162.92943257741985

'Blue'

205.1852821232556

'Red'

...

'Yellow'

249.49148282055643

'White'

278.9283062007153

KeyboardInterrupt: 